# Quantum Phase Estimation on the Hubbard molecule

We would like to study the "Hubbard dimer" molecule, whose Hamiltonian reads:

$$H=-t\sum_{\sigma=\uparrow,\downarrow}\left(c_{1\sigma}^{\dagger}c_{2\sigma}+c_{2\sigma}^{\dagger}c_{1\sigma}\right)-\mu\sum_{i=1,2}\sum_{\sigma=\uparrow,\downarrow}n_{i\sigma}+U\sum_{i=1,2}n_{i\uparrow}n_{i\downarrow}$$

with $n_{i\sigma} = c^\dagger_{i\sigma} c_{i\sigma}$. We will choose $\mu=U/2$ to keep the number of electrons at one per site.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import itertools
from qat.linalg import get_qpu_server
from qat.fermion.hamiltonians import ElectronicStructureHamiltonian
from qat.fermion.phase_estimation import perform_phase_estimation


def make_H(t):
    U = 1.0
    nqbit = 4
    # set h_pq
    # by convention: (i, sig) = 2*i + sig with i: site index and sig: spin index
    hpq = np.zeros((nqbit, nqbit))
    ## hopping
    for sig in [0, 1]:
        hpq[2 * 0 + sig,2 * 1 + sig] = - t
        hpq[2 * 1 + sig,2 * 0 + sig] = - t
    ## chemical potential
    for i in [0, 1]:
        for sig in [0, 1]:
            hpq[2 * i + sig,2 * i + sig] = - U/2

    # h_pqrs: Hubbard interaction
    hpqrs = np.zeros((nqbit, nqbit, nqbit, nqbit))
    for i in [0, 1]:
        for sig in [0, 1]:
            hpqrs[2 * i + sig, 2 * i + 1-sig, 2 * i + sig, 2 * i + 1-sig] = - U

    hamilt = ElectronicStructureHamiltonian(hpq=hpq, hpqrs=hpqrs)
    return hamilt

t = 0.2

hamilt = make_H(t)
eigvals, eigvecs = hamilt.eigenenergies()
E0 = min(eigvals)
print("Exact energy = ", E0)
print("eigvals =", eigvals)


qpu = get_qpu_server()
nqbits_adiab = 6
nqbits_phase = 6
target_energy = E0 + 0.05

print("Guess energy =", target_energy)

for n_adiab_steps, n_trotter_steps in itertools.product([2], [4]):
    energy, prob, eigvec = perform_phase_estimation(hamilt, nqbits_adiab, nqbits_phase, n_adiab_steps, n_trotter_steps,
                                                    target_energy, 0.2, 0.2,
                                                    qpu = qpu,
                                                    n_shots=np.inf, verbose=True)
    print("-> n_adiab_steps =%s, n_trotter=%s, E = %s"%(n_adiab_steps,n_trotter_steps, energy))

Note that one can of course change the ideal QPU for a noisy one.

The circuit used for phase estimation is accessible via the function ``make_phase_estimation_circuit``:

In [ ]:
from qat.fermion.phase_estimation import make_phase_estimation_circuit
n_adiab_steps = 2
n_trotter_steps = 1
circ = make_phase_estimation_circuit(hamilt, nqbits_adiab, nqbits_phase, n_adiab_steps, n_trotter_steps,
                                     target_energy, 0.2, 0.2)

print("Depth of PEA circuit =", len(circ))

## Trotterisation

``qat.fermion.trotterisation`` also provides a trotterization routine (in the Jordan-Wigner representation) which returns a ``QRoutine`` approximating the unitary evolution $\exp(-iHt)$:

In [ ]:
from qat.fermion.trotterisation import trotterisation
from qat.lang.AQASM import Program

n_trotter_steps = 1
final_time = 1

prog = Program()
reg = prog.qalloc(hamilt.hpq.shape[0])
prog.apply(trotterisation(hamilt, n_trotter_steps, final_time), reg)

circ = prog.to_circ()

%qatdisplay circ